## Vision Based Cyclist Travel Lane and Helmet Detection

### Importing Libraries

In [66]:
import numpy as np
import pandas as pd
import cv2

In [67]:
import os
import csv
import pickle
from sklearn.mixture import GaussianMixture

In [68]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score

## Cyclist Travel Lane

### Importing Data

In [69]:
folder1 = r"C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane"
folder2 = r"C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane"

In [70]:
i=0
for filename in os.listdir(folder1):
    #path
    path=os.path.join(folder1,filename)
    a=cv2.imread(path)
    
    #resize image
    resize=(200,200)
    img=cv2.resize(a,resize)
    
    #gray image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #initialise sift descriptor
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    #convert the descriptor array into a dataframe format
    out=pd.DataFrame(descriptors)
    i=i+1
    
    #drop first coloumn as it's the no of feature detected. Not required.
    #append to the csv file
    csv_data=out.to_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane.csv', mode='a', header=False,index=False)

In [71]:
data1 = pd.read_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane.csv',header=None,dtype='uint8')
data1=data1.astype(np.uint8) 
data1

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,117,15,0,2,3,3,5,129,68,129,...,0,44,0,0,0,0,0,0,0,0
1,1,0,0,0,75,113,16,7,71,0,...,38,113,40,0,0,0,0,0,5,86
2,0,1,0,0,1,4,1,0,0,47,...,3,7,5,32,61,100,15,0,0,0
3,10,62,20,43,102,119,33,0,9,118,...,65,86,35,5,0,0,46,119,103,32
4,0,0,9,18,6,0,0,0,8,0,...,0,0,3,84,83,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480149,0,0,1,6,21,19,0,0,2,2,...,4,9,124,28,0,0,0,1,6,99
480150,51,52,10,5,2,0,0,0,141,63,...,0,0,1,6,2,3,2,0,0,0
480151,125,66,3,1,16,61,8,84,113,66,...,0,1,3,1,0,0,1,2,0,0
480152,0,0,0,0,0,0,0,0,0,0,...,29,11,3,1,8,20,21,89,99,6


In [72]:
i=0
for filename in os.listdir(folder2):
    #path
    path=os.path.join(folder2,filename)
    b=cv2.imread(path)
    
    #resize image
    resize=(200,200)
    print(path)
    img=cv2.resize(b,resize)
    
    #gray image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #initialise sift descriptor
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    
    #convert the descriptor array into a dataframe format
    out=pd.DataFrame(descriptors)
    i=i+1
    
    #drop first coloumn as it's the no of feature detected. Not required.
    #append to the csv file
    csv_data=out.to_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane.csv', mode='a', header=False,index=False)

C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1).jpeg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1).png
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (10).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (10).png
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (100).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (100).png
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1000).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1001).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1002).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1003).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1004).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1005).jpg
C:\Users\PAVILION\Downloads\Cyclist lan

In [73]:
data2= pd.read_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane.csv',header=None,dtype='uint8')
data2=data2.astype(np.uint8)
data2

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,8,89,56,17,0,0,0,1,15,130,...,31,17,28,29,15,46,55,39,1,2
1,13,19,14,73,78,3,2,3,2,10,...,18,28,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,4,3,...,114,78,4,0,0,1,1,22,116,95
3,71,83,12,17,14,19,8,44,9,8,...,3,6,22,55,87,4,0,0,3,8
4,0,19,6,0,0,0,0,0,8,60,...,16,30,2,8,68,123,20,7,12,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629769,60,82,8,15,3,6,9,12,128,128,...,1,7,71,82,24,103,91,3,0,1
629770,1,0,6,12,1,0,2,10,42,9,...,0,10,2,4,47,150,26,2,4,7
629771,7,67,150,15,0,1,21,26,57,52,...,1,0,0,0,0,0,0,0,0,0
629772,62,27,1,5,12,7,18,11,1,1,...,0,0,0,0,0,0,0,0,0,0


In [74]:
data=data1.append(data2)

C:\Users\PAVILION\AppData\Local\Temp\ipykernel_456\2617308332.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data1.append(data2)


In [75]:
csv_data=data.to_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\data.csv', mode='a', header=False,index=False)

In [76]:
data

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,117,15,0,2,3,3,5,129,68,129,...,0,44,0,0,0,0,0,0,0,0
1,1,0,0,0,75,113,16,7,71,0,...,38,113,40,0,0,0,0,0,5,86
2,0,1,0,0,1,4,1,0,0,47,...,3,7,5,32,61,100,15,0,0,0
3,10,62,20,43,102,119,33,0,9,118,...,65,86,35,5,0,0,46,119,103,32
4,0,0,9,18,6,0,0,0,8,0,...,0,0,3,84,83,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629769,60,82,8,15,3,6,9,12,128,128,...,1,7,71,82,24,103,91,3,0,1
629770,1,0,6,12,1,0,2,10,42,9,...,0,10,2,4,47,150,26,2,4,7
629771,7,67,150,15,0,1,21,26,57,52,...,1,0,0,0,0,0,0,0,0,0
629772,62,27,1,5,12,7,18,11,1,1,...,0,0,0,0,0,0,0,0,0,0


### Data load

In [77]:
data = pd.read_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\data.csv')

In [78]:
data

,117,15,0,2,3,3.1,5,129,68,129.1,...,0.36,44,0.37,0.38,0.39,0.40,0.41,0.42,0.43,0.44
0,1,0,0,0,75,113,16,7,71,0,...,38,113,40,0,0,0,0,0,5,86
1,0,1,0,0,1,4,1,0,0,47,...,3,7,5,32,61,100,15,0,0,0
2,10,62,20,43,102,119,33,0,9,118,...,65,86,35,5,0,0,46,119,103,32
3,0,0,9,18,6,0,0,0,8,0,...,0,0,3,84,83,0,0,0,0,1
4,50,36,2,0,0,0,0,4,33,24,...,119,134,7,0,0,0,0,0,33,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109922,60,82,8,15,3,6,9,12,128,128,...,1,7,71,82,24,103,91,3,0,1
1109923,1,0,6,12,1,0,2,10,42,9,...,0,10,2,4,47,150,26,2,4,7
1109924,7,67,150,15,0,1,21,26,57,52,...,1,0,0,0,0,0,0,0,0,0
1109925,62,27,1,5,12,7,18,11,1,1,...,0,0,0,0,0,0,0,0,0,0


## GMM/KMeans

In [20]:
pip install seaborn

     -------------------------------------- 288.2/288.2 kB 8.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [79]:
import sklearn.cluster as cluster
import seaborn as sns 

In [81]:
import sklearn.metrics as metrics
for i in range(2,15):
    labels=cluster.KMeans(n_clusters=i,init="k-means++",random_state=200).fit(data).labels_
    print ("Silhouette score for k(clusters) = "+str(i)+" is " +str(metrics.silhouette_score(data,labels,metric="euclidean",sample_size=1000,random_state=200)))

Silhouette score for k(clusters) = 2 is 0.08296981179916989
Silhouette score for k(clusters) = 3 is 0.06062432139395071
Silhouette score for k(clusters) = 4 is 0.055917541588822735
Silhouette score for k(clusters) = 5 is 0.049136742656443806
Silhouette score for k(clusters) = 6 is 0.04616003968772588
Silhouette score for k(clusters) = 7 is 0.047427058566700896
Silhouette score for k(clusters) = 8 is 0.04704103277317715
Silhouette score for k(clusters) = 9 is 0.04664265378054165
Silhouette score for k(clusters) = 10 is 0.04309907548530005
Silhouette score for k(clusters) = 11 is 0.044648749490345564
Silhouette score for k(clusters) = 12 is 0.04580765292215366
Silhouette score for k(clusters) = 13 is 0.04651284764376152
Silhouette score for k(clusters) = 14 is 0.045426851249360065


In [82]:
kmeans = KMeans(n_clusters=9)
kmeans.fit(data)

KMeans(n_clusters=9)

In [83]:
# save the model to disk
filename = 'Kmeans_lane_Model.sav'
pickle.dump(kmeans, open(filename, 'wb'))

In [84]:
i=0
data=[]
k=0

for filename in os.listdir(folder1):
    #path
    path=os.path.join(folder1,filename)
    a=cv2.imread(path)
    
    #resize image
    resize=(200,200)
    print(path)
    img=cv2.resize(a,resize)
    
    #gray image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    sift = cv2.SIFT_create()
    
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    out=pd.DataFrame(descriptors)
    
    #predict values of feature vector with pretrained kmeans
    #ValueError: Buffer dtype mismatch, expected 'float', in order to avoid this dtype=np.double
    
    array_double = np.array(out, dtype=np.double)
    a=kmeans.predict(array_double)
    
    #hist= np.histogram(a,bins=[0,1,2,3,4,5,6,7,8])
    hist= np.histogram(a,bins=[0,1,2,3,4,5,6,7,8,9])
    #append the dataframe into the array in append mode, the array will only have 5 values which will store the values in a row
    data.append(hist[0])
    k=k+1
    
#convert Array to Dataframe and append to the list
Output = pd.DataFrame(data)
#add row class 
Output["Class"] = i 
csv_data=Output.to_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\Finalcyclist.csv', mode='a',header=False,index=False)

C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\1.jpg


C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with featur

C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\10.png
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\11.jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\12.png
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\13 (1).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\13 (1).png
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\13 (2).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\13 (2).png
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\13 (3).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\13 (3).png
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\13 (4).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\13 (4).png
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\13 (5).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\13 (5).png
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\13 (6).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\13 (6).png


C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with featur

C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (1).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (1).png
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (10).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (10).png
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (100).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (101).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (102).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (103).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (104).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (105).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (106).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (107).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (108).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (109).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\14 (11).jpg
C:

...

C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with featur

C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (81).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (82).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (83).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (84).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (85).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (86).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (87).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (88).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (89).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (9).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (90).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (91).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (92).jpg


C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with featur

C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (93).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (94).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (95).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (96).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (97).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (98).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\cyclist_lane\yellow (99).jpg


C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with featur

In [87]:
#performing kmeans prediction of the entire normal_lane dataset with the pretrained kmeans model

#initialising i=1; as its the 2nd class
i=1
data=[]
k=0
for filename in os.listdir(folder2):
    path=os.path.join(folder2,filename)
    a=cv2.imread(path)
    resize=(200,200)
    print(path)
    img=cv2.resize(a,resize)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #sift = cv2.xfeatures2d.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    out=pd.DataFrame(descriptors)
    
    #predict values of feature vector with pretrained kmeans
    #ValueError: Buffer dtype mismatch, expected 'float' but got 'double', in order to avoid this dtype=np.double
    
    array_double = np.array(out, dtype=np.double)
    a=kmeans.predict(array_double)
    hist=np.histogram(a,bins=[0,1,2,3,4,5,6,7,8,9])
    #append the dataframe into the array in append mode, the array will only have 5 values which will store the values in a row
    data.append(hist[0])
    k=k+1
    
#convert Array to Dataframe and append to the list
Output = pd.DataFrame(data)
#add row class 
Output["Class"] = i 
csv_data=Output.to_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\Finalnormal.csv', mode='a',header=False,index=False)

C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1).jpeg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1).png


C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (10).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (10).png
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (100).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (100).png
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1000).jpg


C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with featur

C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1001).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1002).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1003).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1004).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1005).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1006).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1007).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1008).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1009).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (101).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (101).png
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1010).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (1011).jpg
C:\Users\PAVILION\Downloads

...

C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (969).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (97).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (97).png
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (970).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (971).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (972).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (973).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (974).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (975).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (976).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (977).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (978).jpg


C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with featur

C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (979).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (98).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (98).png
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (980).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (981).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (982).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (983).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (984).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (985).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (986).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (987).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (988).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (989).jpg


C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with featur

C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (99).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (99).png
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (990).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (991).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (992).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (993).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (994).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (995).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (996).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (997).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (998).jpg
C:\Users\PAVILION\Downloads\Cyclist lane\normal_lane\normallane (999).jpg


In [88]:
dat1= pd.read_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\Finalcyclist.csv',header=None)
print(dat1)

       0   1   2    3    4   5   6    7   8  9
0     54  68  43   80   83  38  79   44  38  0
1     27  28   1   27   29  24  21    1   2  0
2     29  28  56   29   37  38  37   38  56  0
3     19  30  12   15   19  18  25   23   8  0
4     17  32  23   27   26  12  34   59  44  0
...   ..  ..  ..  ...  ...  ..  ..  ...  .. ..
1495  47  30  58   29   37  24  36   19  33  0
1496  48  31  41   38   28  31  38   35  26  0
1497  12  37  27   38   36  10  51   49  42  0
1498  18  35  26   34   36   9  38   99  49  0
1499  78  87  51  110  122  62  83  101  26  0

[1500 rows x 10 columns]


In [89]:
dat2= pd.read_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\Finalnormal.csv',header=None)
print(dat2)

       0    1   2    3    4   5    6   7   8  9
0     59  109  43   92   87  16  127  18  41  1
1     62   64  47   72   63  63   93  75  39  1
2     34   29  24   21   21  21   27  33  19  1
3     67  112  40  105  108  75   93  54  26  1
4     33   84  39  199  189  21   70  60   3  1
...   ..  ...  ..  ...  ...  ..  ...  ..  .. ..
2995  22   11  85    4    8  21   11  18  64  1
2996  19   23  35   21   17  11   29  65  32  1
2997  32   40  38   31   39  13   32  65  18  1
2998  96   67  71   52   64  42   59  30  77  1
2999  79   69  72   59   68  37   60  36  26  1

[3000 rows x 10 columns]


In [90]:
A=dat1.append(dat2)
csv_data=A.to_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\classifier1.csv', mode='a',header=False,index=False)

C:\Users\PAVILION\AppData\Local\Temp\ipykernel_456\630241772.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  A=dat1.append(dat2)


In [91]:
A

,0,1,2,3,4,5,6,7,8,9
0,54,68,43,80,83,38,79,44,38,0
1,27,28,1,27,29,24,21,1,2,0
2,29,28,56,29,37,38,37,38,56,0
3,19,30,12,15,19,18,25,23,8,0
4,17,32,23,27,26,12,34,59,44,0
...,...,...,...,...,...,...,...,...,...,...
2995,22,11,85,4,8,21,11,18,64,1
2996,19,23,35,21,17,11,29,65,32,1
2997,32,40,38,31,39,13,32,65,18,1
2998,96,67,71,52,64,42,59,30,77,1


In [92]:
x = A.iloc[:,0:9].values
x

array([[54, 68, 43, ..., 79, 44, 38],
       [27, 28,  1, ..., 21,  1,  2],
       [29, 28, 56, ..., 37, 38, 56],
       ...,
       [32, 40, 38, ..., 32, 65, 18],
       [96, 67, 71, ..., 59, 30, 77],
       [79, 69, 72, ..., 60, 36, 26]], dtype=int64)

In [93]:
y = A.iloc[:,9].values
y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [94]:
from sklearn.preprocessing import StandardScaler
xs = StandardScaler()
x_pca = xs.fit_transform(x)
x_pca

array([[ 0.42093068,  0.61692671,  0.27339482, ...,  0.80135948,
        -0.13422715,  0.38742255],
       [-0.59548679, -0.63489734, -1.67104792, ..., -0.93537409,
        -1.97981476, -1.4047663 ],
       [-0.52019661, -0.63489734,  0.87524615, ..., -0.45627518,
        -0.39175101,  1.28351698],
       ...,
       [-0.40726133, -0.25935012,  0.04191354, ..., -0.60599359,
         0.76710633, -0.60823792],
       [ 2.00202453,  0.58563111,  1.56968999, ...,  0.20248583,
        -0.73511614,  2.32896048],
       [ 1.36205797,  0.64822231,  1.61598624, ...,  0.23242952,
        -0.47759229, -0.20997373]])

In [95]:
from sklearn.decomposition import PCA

In [96]:
pca = PCA(n_components=None)
pca.fit(x_pca)

PCA()

In [97]:
cycle_pca = pca.transform(x_pca)
cycle_pca

array([[ 1.78000965, -0.25360027, -0.42071438, ...,  0.27059254,
        -0.1763028 ,  0.0179871 ],
       [-2.44868537, -1.76463343, -0.37302312, ...,  0.07035271,
         0.07226298, -0.02974566],
       [-0.53053836,  1.72922673, -0.23821614, ..., -0.06103061,
        -0.06346145,  0.18035032],
       ...,
       [-1.08470382,  0.0415386 ,  0.22608404, ...,  0.06717781,
         0.3562058 ,  0.06853378],
       [ 2.15733757,  2.17417027, -0.38549579, ...,  0.29282699,
         0.33837544,  0.14310032],
       [ 1.65417639,  0.33084932, -0.10988456, ...,  0.15169794,
         0.35672452,  0.07124611]])

In [98]:
cycle_pca = pd.DataFrame(cycle_pca)
cycle_pca

,0,1,2,3,4,5,6,7,8
0,1.780010,-0.253600,-0.420714,0.065418,-0.267012,0.123546,0.270593,-0.176303,0.017987
1,-2.448685,-1.764633,-0.373023,1.384618,-0.329200,-0.004219,0.070353,0.072263,-0.029746
2,-0.530538,1.729227,-0.238216,0.586315,-0.288425,0.565804,-0.061031,-0.063461,0.180350
3,-2.526895,-0.986772,-0.228563,0.510492,-0.022329,0.006348,-0.339088,0.157331,0.022988
4,-1.599104,0.676917,-0.216798,-0.934120,-0.642610,-0.160304,-0.080195,0.053474,-0.064381
...,...,...,...,...,...,...,...,...,...
4495,-1.731777,3.156842,-1.067028,1.332146,0.668234,0.552281,-0.061804,0.155495,0.104112
4496,-1.862565,0.782070,0.126471,-0.967091,0.215197,-0.052985,-0.128628,-0.006277,-0.071555
4497,-1.084704,0.041539,0.226084,-0.883255,0.691237,-0.096591,0.067178,0.356206,0.068534
4498,2.157338,2.174170,-0.385496,1.541543,-0.284752,-1.041988,0.292827,0.338375,0.143100


In [99]:
cycle_pca.shape

(4500, 9)

In [100]:
print(pca.explained_variance_ratio_) 

[0.56412031 0.15313847 0.11018447 0.09864675 0.02686202 0.02107576
 0.01259877 0.00919529 0.00417817]


In [101]:
pca = PCA(n_components=9)
pca.fit(x_pca)

PCA(n_components=9)

In [102]:
# save the model to disk
filename = 'PCA_cycle_Model.sav'
pickle.dump(kmeans, open(filename, 'wb'))

In [103]:
cycle_pca = pca.transform(x_pca)
cycle_pca

array([[ 1.78000965, -0.25360027, -0.42071438, ...,  0.27059254,
        -0.1763028 ,  0.0179871 ],
       [-2.44868537, -1.76463343, -0.37302312, ...,  0.07035271,
         0.07226298, -0.02974566],
       [-0.53053836,  1.72922673, -0.23821614, ..., -0.06103061,
        -0.06346145,  0.18035032],
       ...,
       [-1.08470382,  0.0415386 ,  0.22608404, ...,  0.06717781,
         0.3562058 ,  0.06853378],
       [ 2.15733757,  2.17417027, -0.38549579, ...,  0.29282699,
         0.33837544,  0.14310032],
       [ 1.65417639,  0.33084932, -0.10988456, ...,  0.15169794,
         0.35672452,  0.07124611]])

In [104]:
cycle_pca.shape

(4500, 9)

In [105]:
print(pca.explained_variance_ratio_) 

[0.56412031 0.15313847 0.11018447 0.09864675 0.02686202 0.02107576
 0.01259877 0.00919529 0.00417817]


In [106]:
cycle_pca = pd.DataFrame(cycle_pca)
cycle_pca

,0,1,2,3,4,5,6,7,8
0,1.780010,-0.253600,-0.420714,0.065418,-0.267012,0.123546,0.270593,-0.176303,0.017987
1,-2.448685,-1.764633,-0.373023,1.384618,-0.329200,-0.004219,0.070353,0.072263,-0.029746
2,-0.530538,1.729227,-0.238216,0.586315,-0.288425,0.565804,-0.061031,-0.063461,0.180350
3,-2.526895,-0.986772,-0.228563,0.510492,-0.022329,0.006348,-0.339088,0.157331,0.022988
4,-1.599104,0.676917,-0.216798,-0.934120,-0.642610,-0.160304,-0.080195,0.053474,-0.064381
...,...,...,...,...,...,...,...,...,...
4495,-1.731777,3.156842,-1.067028,1.332146,0.668234,0.552281,-0.061804,0.155495,0.104112
4496,-1.862565,0.782070,0.126471,-0.967091,0.215197,-0.052985,-0.128628,-0.006277,-0.071555
4497,-1.084704,0.041539,0.226084,-0.883255,0.691237,-0.096591,0.067178,0.356206,0.068534
4498,2.157338,2.174170,-0.385496,1.541543,-0.284752,-1.041988,0.292827,0.338375,0.143100


In [107]:
B=pd.concat([cycle_pca, pd.DataFrame(y)],axis=1)
B

,0,1,2,3,4,5,6,7,8,0
0,1.780010,-0.253600,-0.420714,0.065418,-0.267012,0.123546,0.270593,-0.176303,0.017987,0
1,-2.448685,-1.764633,-0.373023,1.384618,-0.329200,-0.004219,0.070353,0.072263,-0.029746,0
2,-0.530538,1.729227,-0.238216,0.586315,-0.288425,0.565804,-0.061031,-0.063461,0.180350,0
3,-2.526895,-0.986772,-0.228563,0.510492,-0.022329,0.006348,-0.339088,0.157331,0.022988,0
4,-1.599104,0.676917,-0.216798,-0.934120,-0.642610,-0.160304,-0.080195,0.053474,-0.064381,0
...,...,...,...,...,...,...,...,...,...,...
4495,-1.731777,3.156842,-1.067028,1.332146,0.668234,0.552281,-0.061804,0.155495,0.104112,1
4496,-1.862565,0.782070,0.126471,-0.967091,0.215197,-0.052985,-0.128628,-0.006277,-0.071555,1
4497,-1.084704,0.041539,0.226084,-0.883255,0.691237,-0.096591,0.067178,0.356206,0.068534,1
4498,2.157338,2.174170,-0.385496,1.541543,-0.284752,-1.041988,0.292827,0.338375,0.143100,1


In [108]:
csv_data=B.to_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\FinalPCAClassifier1.csv', mode='a',header=False,index=False)

## Training ,Testing & Classification

In [109]:
data= pd.read_csv(r'C:\Users\PAVILION\Downloads\Cyclist lane\FinalPCAClassifier1.csv',header=None)
data

,0,1,2,3,4,5,6,7,8,9
0,1.780010,-0.253600,-0.420714,0.065418,-0.267012,0.123546,0.270593,-0.176303,0.017987,0
1,-2.448685,-1.764633,-0.373023,1.384618,-0.329200,-0.004219,0.070353,0.072263,-0.029746,0
2,-0.530538,1.729227,-0.238216,0.586315,-0.288425,0.565804,-0.061031,-0.063461,0.180350,0
3,-2.526895,-0.986772,-0.228563,0.510492,-0.022329,0.006348,-0.339088,0.157331,0.022988,0
4,-1.599104,0.676917,-0.216798,-0.934120,-0.642610,-0.160304,-0.080195,0.053474,-0.064381,0
...,...,...,...,...,...,...,...,...,...,...
4495,-1.731777,3.156842,-1.067028,1.332146,0.668234,0.552281,-0.061804,0.155495,0.104112,1
4496,-1.862565,0.782070,0.126471,-0.967091,0.215197,-0.052985,-0.128628,-0.006277,-0.071555,1
4497,-1.084704,0.041539,0.226084,-0.883255,0.691237,-0.096591,0.067178,0.356206,0.068534,1
4498,2.157338,2.174170,-0.385496,1.541543,-0.284752,-1.041988,0.292827,0.338375,0.143100,1


In [110]:
#assigning x the columns from 1 to 128 for training
x = data.iloc[:,0:9].values
print("X values")
print(x)

#assigning y with the column "Class" as target variable
y = data.iloc[:,9]
print("Y values")
print(y)

X values
[[ 1.78000965 -0.25360027 -0.42071438 ...  0.27059254 -0.1763028
   0.0179871 ]
 [-2.44868537 -1.76463343 -0.37302312 ...  0.07035271  0.07226298
  -0.02974566]
 [-0.53053836  1.72922673 -0.23821614 ... -0.06103061 -0.06346145
   0.18035032]
 ...
 [-1.08470382  0.0415386   0.22608404 ...  0.06717781  0.3562058
   0.06853378]
 [ 2.15733757  2.17417027 -0.38549579 ...  0.29282699  0.33837544
   0.14310032]
 [ 1.65417639  0.33084932 -0.10988456 ...  0.15169794  0.35672452
   0.07124611]]
Y values
0       0
1       0
2       0
3       0
4       0
       ..
4495    1
4496    1
4497    1
4498    1
4499    1
Name: 9, Length: 4500, dtype: int64


In [111]:
#Dataset split into train and test with 80% Training and 20% Testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.20, random_state=0)

## Decision tree

In [112]:
from sklearn.tree import DecisionTreeClassifier

In [113]:
#Decision Tree Classifier
from sklearn.metrics import accuracy_score
#Assign model with Decision Tree classifier


In [114]:
model1 = DecisionTreeClassifier(max_depth=13)
filename = 'model1.sav'
pickle.dump(kmeans, open(filename, 'wb'))


In [115]:
y

0       0
1       0
2       0
3       0
4       0
       ..
4495    1
4496    1
4497    1
4498    1
4499    1
Name: 9, Length: 4500, dtype: int64

In [116]:
#training the model with the Training Variables 
model1.fit(x_train, y_train)
#predicting the traget variable using testing variables
y_pred1 = model1.predict(x_test)
#Results
print("Decision Tree Results")
print("Decision Tree Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")


Decision Tree Results
Decision Tree Accuracy:  81.0 %


In [122]:
from sklearn.preprocessing import StandardScaler
test1S = StandardScaler()
test1 = test1S.fit_transform(Output)

C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\PAVILION\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [123]:
test1

array([[ 0.43070028,  1.76985363,  0.19336307, ..., -1.46010637,
         0.60229399,  0.        ],
       [ 0.54694518,  0.33777873,  0.37854717, ...,  1.18933642,
         0.5002045 ,  0.        ],
       [-0.53800724, -0.7760573 , -0.68626142, ..., -0.76288459,
        -0.52069046,  0.        ],
       ...,
       [-0.61550384, -0.42599455, -0.03811706, ...,  0.72452189,
        -0.5717352 ,  0.        ],
       [ 1.8643874 ,  0.43325039,  1.48965178, ..., -0.90232894,
         2.43990491,  0.        ],
       [ 1.20566629,  0.49689816,  1.53594781, ..., -0.62344023,
        -0.16337722,  0.        ]])

In [ ]:
DT

array([[-5.90835798e-17,  2.64123640e-17, -2.89210285e-17,
         1.47605955e-16,  3.89593546e-17,  1.45873670e-17]])

## Random Forest

In [126]:
from sklearn.ensemble import RandomForestClassifier

In [127]:
para = [] # list to save all parameter values to plot
acc = [] # list to save all accuracy values to plot
for i in range(10,500,10):
  rfc = RandomForestClassifier(n_estimators=i)
  rfc = rfc.fit(x_train,y_train)
  y_pred1 = rfc.predict(x_test)
  para.append(i)
  acc.append(accuracy_score(y_test, y_pred1)*100)

In [ ]:
para[acc.index(max(acc))], max(acc)

(30, 84.88888888888889)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
rfc = RandomForestClassifier(n_estimators=350)
rfc = rfc.fit(x_train,y_train)
y_pred1 = rfc.predict(x_test)
#Results
print("RandomForestClassifier Results")
print("RandomForestClassifier Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("CM = \n",confusion_matrix(y_test, y_pred1))

RandomForestClassifier Results
RandomForestClassifier Accuracy:  84.0 %
CM = 
 [[166 135]
 [  9 590]]


In [136]:
file = 'HRF_Model.sav'
pickle.dump(kmeans, open(file, 'wb'))

## KNN

In [132]:
from sklearn.neighbors import KNeighborsClassifier

In [133]:
para = []
acc = []
for i in range(1,50,1):
  knc = KNeighborsClassifier(n_neighbors= i,p = 3)
  knc = knc.fit(x_train,y_train)
  y_pred1 = knc.predict(x_test)
  para.append(i)
  acc.append(accuracy_score(y_test, y_pred1)*100)

In [134]:
knc = KNeighborsClassifier(n_neighbors= 7,p = 3)
knc = knc.fit(x_train,y_train)

In [135]:
y_pred1 = knc.predict(x_test)
#Results
print("KNN Results")
print("KNN Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("CM = \n",confusion_matrix(y_test, y_pred1))

KNN Results
KNN Accuracy:  75.66666666666667 %
CM = 
 [[151 150]
 [ 69 530]]
